In [1]:
# Importing libraries

import pandas as pd

In [4]:
# Reading the master spreadsheet

df = pd.read_csv("https://raw.githubusercontent.com/atreish/yukatzbot/main/tests/masters_spreadsheet_with_unique_intent%20(11).csv")
df.head()

,question,answer,primary_entities,secondary_entities,primary_types,secondary_types,unique_intent
0,What are the admission requirements and how do...,Please submit the following required document ...,admission requirements,application process,"['faculty', 'academic programs and research', ...","['faculty', 'academic programs and research', ...",admission_requirements_application_process
1,What are the program's curriculum and research...,Please send student program curriculum via web...,programs curriculum,research opportunities,academic programs and research,- academic programs and research,programs_curriculum_research_opportunities
2,"What is the cost of the program, and are there...",The STEM Fellows Scholarship is awarded to qua...,cost of the program,scholarships financial aid options,miscellaneous,financial aid options,cost_of_the_program_scholarships_financial_aid...
3,"What is the application deadline, and when can...","For the Fall 2023 semester, our priority deadl...",application deadline,admission team,miscellaneous,faculty and experts,application_deadline_admission_team
4,"What is the acceptance rate for the program, a...",Some of our STEM programs do have pre-requisit...,acceptance rate,program,"['university and campus life', 'miscellaneous']","['academic programs and research', 'career and...",acceptance_rate_program


In [6]:
# Function to generate nlu.yml file for each row
def generate_yaml_row(question, unique_intent):
    return f"- intent: {unique_intent}\n  examples: |\n    - {question}"

# Convert DataFrame to YAML format
yaml_content = "\n".join(df.apply(lambda row: generate_yaml_row(row["question"], row["unique_intent"]), axis=1))

# Save the YAML content to a file
with open("nlu.yaml", "w") as file:
    file.write(yaml_content)

In [9]:
# Function to generate stories.yml file for each row

def generate_stories_yaml_row(unique_intent):
    yaml_content = (
        f"- story: {unique_intent}\n"
        f"  steps:\n"
        f"  - intent: {unique_intent}\n"
        f"  - action: utter_{unique_intent}\n"
    )
    return yaml_content

# Convert unique intents to YAML format
yaml_content = "\n".join(df.apply(lambda row: generate_stories_yaml_row(row["unique_intent"]), axis=1))

# Save the YAML content to a file
with open("stories.yaml", "w") as file:
    file.write(yaml_content)

In [12]:
# Functions to generate domain.yml file for each row

# Function to generate intents section in YAML format
def generate_intents_section(df):
    intents = "\n  - ".join(df['unique_intent'])
    return f"intents:\n  - {intents}\n\n"

# Function to generate actions section in YAML format
def generate_actions_section(df):
    actions = "\n  - ".join([f"utter_{intent}" for intent in df['unique_intent']])
    return f"actions:\n  - {actions}\n\n"

# Function to generate responses section in YAML format
def generate_responses_section(df):
    responses = "responses:\n"
    for _, row in df.iterrows():
        responses += f"  utter_{row['unique_intent']}:\n    - text: {row['answer']}\n\n"
    return responses

# Generate YAML content
yaml_content = generate_intents_section(df)
yaml_content += generate_actions_section(df)
yaml_content += generate_responses_section(df)

# Save the YAML content to a file
with open("domain.yaml", "w") as file:
    file.write(yaml_content)